In [1]:
from models_utils import t5_model, gpt2_model
from utils import chat_answer

C:\Users\minnb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load INSTRUCTOR_Transformer
max_seq_length  512


In [2]:
import pandas as pd

# Path to the CSV file
csv_file_path = '../data/qa/llm_test_1000.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to verify it loaded correctly
print(df.head())

                                            Question  \
0                  # of caloriesfor 1 cand yginger ?   
1         is the kotobukiya superman a resin statue?   
2                                Does it have word ?   
3  Is this the 2012 version with the mounted boar...   
4  I have a Maine Coon, (beautiful) a shelter cat...   

                                             Answers  
0  Hi...Melissa.  Each Natural Ginger Tummy Drops...  
1  I am unsure I do not see specifics|Its a PVC v...  
2  It has Office 4.0.1|It does not have Microsoft...  
3  I bought this in 2013 so I assume so. It's ver...  
4  Does she try to"fish" things out?  You could p...  


In [6]:
df.iloc[0]['Question']

'# of caloriesfor 1 cand yginger ?'

In [8]:
from rouge_score import rouge_scorer
# Initialize the scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])

In [7]:
#choose model
llm = t5_model(temp = 0,rep = 1.5)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
scores = []

for i in range(5):
    reference = df.iloc[i]['Answers']
    prompt_question = df.iloc[i]['Question']
    candidate_full = chat_answer(prompt_question, llm)
    candidate = candidate_full['answer']
    score = scorer.score(reference, candidate)
    scores.append(score)
    
# example score
print(scores[0])
#It should return average ROGUE-1



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Test prompt for NLP Amazon sales chatbot.
        Type\": \"Oily, Combination, Dry, Normal\", \"Material Feature\": \"Organic, Natural\", \"Item Volume\": \"30 Milliliters\", \"Active Ingredients\": \"hyaluronic_acid\", \"Is Discontinued By Manufacturer\": \"No\", \"Package Dimensions\": \"5.5 x 1.25 x 1.25 inches; 3.21 Ounces\", \"UPC\": \"816554010859\", \"Manufacturer\": \"Asara New York\"}", "parent_asin": "B07FXWHFRF", "bought_together": null, "subtitle": null, "author": null}

more\", \"Material Feature\": \"Non-Toxic\", \"Age Range (Description)\": \"Adult\", \"Number of Items\": \"1\", \"Liquid Volume\": \"11 Milliliters\", \"Is Discontinued By Manufacturer\": \"No\", \"Product Dimensions\": \"1 x 0.5 x 2.5 inches; 0.32 Ounces\", \"Item model number\": \"BUTTER-2692\", \"Department\": \"Womens\", \"UPC\": \"851847002692

C:\Users\minnb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.
{'rouge1': Score(precision=0.3333333333333333, recall=0.004310344827586207, fmeasure=0.00851063829787234), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.3333333333333333, recall=0.004310344827586207, fmeasure=0.00851063829787234), 'rougeLsum': Score(precision=0.3333333333333333, recall=0.004310344827586207, fmeasure=0.00851063829787234)}


In [16]:
def Average(list): 
    return sum(list) / len(list)

In [17]:
precisions = recall = fmeasure = []
for sco in scores:
    precisions.append(sco['rouge1'][0])
    recall.append(sco['rouge1'][1])
    fmeasure.append(sco['rouge1'][2])

pre_avg = Average(precisions)
recall_avg = Average(recall)
f_avg = Average(fmeasure)
overall_avg = (pre_avg + recall_avg + f_avg) / 3
print(f"ROUGE-1 scores:\nAverage Precision: {pre_avg}\nAverage Recall: {recall_avg}\nAverage fmeasure: {f_avg}\nAverage overall: {overall_avg}")

ROUGE-1 scores:
Average Precision: 0.15304438758411162
Average Recall: 0.15304438758411162
Average fmeasure: 0.15304438758411162
Average overall: 0.15304438758411162
